In [ ]:
#importing the essential libraries
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing import image

In [ ]:
train_set='/content/sample_data/'
val_set='/content/sample_data/'
test_set='/content/sample_data/'

In [ ]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1
)

In [ ]:
validation_datagen= image.ImageDataGenerator(    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1)

In [ ]:
test_datagen= image.ImageDataGenerator(    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1)

In [ ]:
# image addressing
train_generator = train_datagen.flow_from_directory(
    train_set,
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'categorical')

Found 1500 images belonging to 4 classes.


In [ ]:
validation_generator = validation_datagen.flow_from_directory(
    val_set,
    target_size = (224,224),
    batch_size = 16,
    shuffle=True,
    class_mode = 'categorical')

Found 1500 images belonging to 4 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_set,
    target_size = (224,224),
    batch_size = 16,
    class_mode = 'categorical')

Found 1500 images belonging to 4 classes.


In [ ]:
train_generator.class_indices

{'.ipynb_checkpoints': 0,
 '3_esophagitis_test': 1,
 '3_esophagitis_train': 2,
 '3_esophagitis_val': 3}

In [ ]:
from tensorflow.keras import regularizers
from keras.regularizers import l2

In [ ]:
base_model = tf.keras.applications.EfficientNetB2(weights='imagenet', input_shape=(224,224,3), include_top=False)

In [ ]:
for layer in base_model.layers:
    layer.trainable=False
model = Sequential()
model.add(base_model)
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(GaussianNoise(0.35))
model.add(GlobalAveragePooling2D())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(GaussianNoise(0.35))
model.add(Dropout(0.2))
model.add(Dense(4, activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetb2 (Functional)  (None, 7, 7, 1408)       7768569   
                                                                 
 gaussian_noise_2 (GaussianN  (None, 7, 7, 1408)       0         
 oise)                                                           
                                                                 
 global_average_pooling2d_2   (None, 1408)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_5 (Dense)             (None, 256)               360704    
                                                                 
 batch_normalization_95 (Bat  (None, 256)              1024      
 chNormalization)                                                
                                                      

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy', metrics=['accuracy','Precision','Recall','AUC'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=20, #steps_per_epoch= training_images/batch_size= 3200/32<=200
    epochs = 20, #own choice
    validation_data = validation_generator
)

Epoch 1/20
20/20 [==============================] - 252s 12s/step - loss: 1.3144 - accuracy: 0.5562 - precision: 0.6061 - recall: 0.5000 - auc: 0.7902 - val_loss: 1.0751 - val_accuracy: 0.5360 - val_precision: 0.5474 - val_recall: 0.4967 - val_auc: 0.8160
Epoch 2/20
20/20 [==============================] - 245s 13s/step - loss: 1.1159 - accuracy: 0.6531 - precision: 0.6952 - recall: 0.6344 - auc: 0.8581 - val_loss: 0.9032 - val_accuracy: 0.6240 - val_precision: 0.6669 - val_recall: 0.5380 - val_auc: 0.8606
Epoch 3/20
20/20 [==============================] - 245s 13s/step - loss: 0.8290 - accuracy: 0.7156 - precision: 0.7534 - recall: 0.6969 - auc: 0.9086 - val_loss: 0.6345 - val_accuracy: 0.7927 - val_precision: 0.8336 - val_recall: 0.7347 - val_auc: 0.9344
Epoch 4/20
20/20 [==============================] - 245s 13s/step - loss: 0.7745 - accuracy: 0.7312 - precision: 0.7584 - recall: 0.7063 - auc: 0.9214 - val_loss: 0.5994 - val_accuracy: 0.7920 - val_precision: 0.8247 - val_recall: 0

Transfer Learning

In [ ]:
InceptionV3_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False , input_shape=(224,224,3))

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential

In [ ]:
#Thr last 15 layers fine tune
for layer in InceptionV3_model.layers[:-15]:
  layer.trainable = False

In [ ]:
x= InceptionV3_model.output
x=GlobalAveragePooling2D()(x)
x=Flatten()(x)
x=Dense(units=512,activation='relu')(x)
x=Dropout(0.3)(x)
x=Dense(units=512,activation='relu')(x)
x=Dropout(0.3)(x)
output = Dense(units=5, activation='softmax')(x)
model=Model(InceptionV3_model.input,output)

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 111, 111, 32  864         ['input_4[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_96 (BatchN  (None, 111, 111, 32  96         ['conv2d_94[0][0]']              
 ormalization)                  )                                                           

In [ ]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),optimizer="adam",metrics=['accuracy'])

In [ ]:
epochs=5
history = model.fit(
train_generator,
validation_data=validation_generator,
epochs=epochs
)

Epoch 1/5


InvalidArgumentError: ignored

In [ ]:
epochs=5
history3=model.fit(train_ds1,validation_data=val_ds1,epochs=epochs)

In [ ]:
acc = history3.history['accuracy']

val_acc history3.history['val_accuracy']

loss = history3.history['loss']

val_loss = history3.history['val_loss']

epochs_range = range (epochs)

plt.figure(figsize=(8, 8))

plt.subplot(1, 2, 1)

plt.plot(epochs_range, acc, label='Training Accuracy')

plt.plot(epochs_range, val_acc, label='Validation Accuracy')

plt.legend(loc='lower right')

plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)

plt.plot(epochs_range, loss, label='Training Loss')

plt.plot(epochs_range, val_loss, label='Validation Loss')

plt.legend (loc='upper right')

plt.title('Training and Validation Loss')

plt.show()

In [ ]:
#RESNET
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.optimizers import SGD
from keras.callbacks import Earlystopping
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping, ModelCheckpoint

from google.colab.patches Import cv2 Imshow

from keras layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv20, AveragePooling20, MaxPooling20, GlobalMaxPooling, MaxP

from keras preprocessing import image

from keras.initializers import glorot uniform

img_height, img_width=64,64

num classes 5

#If imagenet weights are being loaded,

#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
base_model = tf.keras.applications.resnet50. ResNet50 (weights=None, include_top=False, Input_shape=(img_height, img_width, 3))

In [ ]:
x= base_model.output

x = GlobalAveragePooling2D()(x)
x= Dropout (0.7)(x)

predictions = Dense (num_classes, activation= 'softmax')(x)
model = Model(inputs =base_model. input, outputs =predictions)

from keras.optimizers import Adam

model.compile(loss=tf.keras. losses. SparseCategoricalCrossentropy(), optimizer='adam', metrics=["accuracy"])

val ds1= tf.keras.utils.image_dataset_from_directory( data_dir,

validation_split=0.2,

subset ="validation",

seed=123,

image_size=(img_height, img_width),

batch_size=batch_size)
train_ds1 = tf.keras.utils.image_dataset from directory(

data_dir,

validation_split=0.2,

subset="training",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size)

epochs 5

history3 = model.fit(

train_ds1,

validation_data=val_ds1,

epochs=epochs)

acc = history3.history['accuracy']

val_acc = history3. history['val_accuracy']

loss = history3.history['loss']

val_loss = history3.history['val_loss']

epochs_range= range(epochs)

plt.figure(figsize=(8, 8))

plt.subplot(1, 2, 1)

plt.plot(epochs_range, acc, label="Training Accuracy")

plt.plot(epochs_range, val_acc, label="Validation Accuracy")

plt.legend(loc= "lower right")

plt.title("Training and Validation Accuracy")

plt.subplot(1, 2, 2)

plt.plot(epochs_range, loss, label='Training Loss')

plt.plot(epochs_range, val_loss, label="validation Loss")

plt.legend(loc='upper right')

plt.title('Training and Validation Loss')

plt.show()